# Ejercicios de Selenium : La casa del libro.

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [10]:
# Configuración inicial
navegador = webdriver.Chrome()
navegador.get("https://www.casadellibro.com")
navegador.maximize_window()

# Aceptar cookies
try:
    WebDriverWait(navegador, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#onetrust-accept-btn-handler'))
    ).click()
except Exception as ex:
    print("No se pudo aceptar cookies:", ex)

# Entrada del usuario
autor_usuario = input("Introduce el nombre del autor que quieres buscar: ")
busqueda = navegador.find_element(By.XPATH, '//*[@id="empathy-search"]')
busqueda.send_keys(autor_usuario)
busqueda.send_keys(Keys.ENTER)

# Esperar que los resultados se carguen
try:
    WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'roller'))
    )
except Exception as ex:
    print("No se encontraron resultados visibles:", ex)
    navegador.quit()

# Contar los resultados disponibles
resultados_disponibles = len(navegador.find_elements(By.XPATH, "//div[@class='roller']//div"))
print(f"Resultados disponibles: {resultados_disponibles}")

if resultados_disponibles == 0:
    print("No se encontraron resultados. Verifica tu término de búsqueda o ajusta el código.")
    navegador.quit()

# Diccionario para guardar resultados
info_libros = {'Titulo': [], 'Autor': [], 'Precio': [], 'Editorial': []}

# Iterar según los resultados disponibles
for i in range(1, resultados_disponibles + 1):
    try:
        # Seleccionar producto
        WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"(//div[@class='roller']//div)[{i}]"))
        ).click()
        sleep(2)

        # Extraer datos
        titulo = navegador.find_element(By.CSS_SELECTOR, '.titleProducto').text
        autor = navegador.find_element(By.CSS_SELECTOR, '.f-serif a').text
        precio = navegador.find_element(By.CSS_SELECTOR, '.info-price p').text
        editorial = navegador.find_element(By.CSS_SELECTOR, '.area-2 p').text

        # Guardar solo si coincide con el autor
        if autor_usuario.lower() in autor.lower():
            info_libros['Titulo'].append(titulo)
            info_libros['Autor'].append(autor)
            info_libros['Precio'].append(precio)
            info_libros['Editorial'].append(editorial)

        # Volver a la página principal de resultados
        navegador.back()
        WebDriverWait(navegador, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'roller'))
        )
    except Exception as ex:
        print(f"Error al analizar el artículo {i}: {ex}")
        navegador.back()
        sleep(2)
        WebDriverWait(navegador, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'roller'))
        )   


Resultados disponibles: 0
No se encontraron resultados. Verifica tu término de búsqueda o ajusta el código.


In [11]:
# Crear DataFrame y mostrar
df = pd.DataFrame(info_libros)
print(df)

# Cerrar navegador
navegador.quit()

Empty DataFrame
Columns: [Titulo, Autor, Precio, Editorial]
Index: []
